In [17]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

run = exp.start_logging()

In [18]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "Udactiy-Project-Cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# Choose a name for your CPU cluster

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": choice(0.5,1.0, 1.5),
    "--max_iter": choice(50, 75, 100, 125, 150)
  })


# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=aml_compute,
                      environment=sklearn_env)


# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                                    run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name="Accuracy",
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=20,
                                    max_concurrent_runs=4,
                                    #estimator=est,
                                    )

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
from azureml.core.experiment import Experiment

experiment = Experiment(ws, "test_hyperdrive_experiment")


In [5]:
hdr = experiment.submit(config=hyperdrive_config)
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hdr.get_best_run_by_primary_metric()

best_run_metrics = best_run.get_metrics()

print(best_run_metrics)

{'Regularization Strength:': 1.0, 'Max iterations:': 75, 'Accuracy': 0.9116240573845871}


In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url_paths=["https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"]
ds = Dataset.Tabular.from_delimited_files(path=url_paths)
    

In [11]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
predict_col = pd.DataFrame(y)
full_data = pd.concat([x,predict_col],axis=1)
# Save training data to a csv file
full_data.to_csv("bankmarket_training.csv")
# Read training data from CSV File
data_store = ws.get_default_datastore()
data_store.upload(src_dir='./',target_path='training/')
final_data = Dataset.Tabular.from_delimited_files(path = [(data_store, 'training/bankmarket_training.csv')])

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 14 files
Uploading ./.amlignore
Uploaded ./.amlignore, 1 files out of an estimated total of 14
Uploading ./.amlignore.amltmp
Uploaded ./.amlignore.amltmp, 2 files out of an estimated total of 14
Uploading ./AutoML evaluation.JPG
Uploaded ./AutoML evaluation.JPG, 3 files out of an estimated total of 14
Uploading ./CODEOWNERS
Uploaded ./CODEOWNERS, 4 files out of an estimated total of 14
Uploading ./conda_dependencies.yml
Uploaded ./conda_dependencies.yml, 5 files out of an estimated total of 14
Uploading ./Delete.JPG
Uploaded ./Delete.JPG, 6 files out of an estimated total of 14
Uploading ./Hyperrun evaluation.JPG
Uploaded ./Hyperrun evaluation.JPG, 7 files out of an estimated total of 14
Uploading ./README.md
Uploaded ./README.md, 8 files out of an estimated total of 14
Uploading ./bankmarket_training.csv
Uploaded ./bankmarket_training.csv, 9 files out of an estimated total of 14
Uploading ./train.py
Uploaded ./train.py, 10 files out of an estimated total of 1

In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="AUC_weighted",
    training_data=final_data,
    label_column_name="y",
    n_cross_validations=2,
    compute_target = aml_compute,
    enable_early_stopping = True)

In [13]:
# Submit your automl run

### YOUR CODE HERE ###
experiment = Experiment(ws, "automl_test_experiment")
automl_run = exp.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on Udactiy-Project-Cluster with default configuration
Running on remote compute: Udactiy-Project-Cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_a4654f6f-450b-459f-9eea-0818dcbfee1f,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

In [15]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)



Run(Experiment: udacity-project,
Id: AutoML_a4654f6f-450b-459f-9eea-0818dcbfee1f_49,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None, criterion='gini', max_depth=None, max_features='auto', max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1, oob_score=False, random_state=None, verbose=0, warm_start=False))], verbose=False))], flatten_transform=None, weights=[0.2, 0.0666

## Clean up cluster

In [16]:
aml_compute.delete()